In [1]:
%pip install yfinance
%pip install tqdm

  Obtaining dependency information for yfinance from https://files.pythonhosted.org/packages/e9/43/93ea65227c938a0a4a8925d7f054b050ea743044b887170da9eca210635a/yfinance-0.2.28-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 2.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import bs4 as bs
import requests
import yfinance as yf
import datetime
import numpy as np
from tqdm.notebook import tqdm, trange
import pandas as pd
import math
from dateutil.relativedelta import relativedelta

In [24]:
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})

def SP500_scraper(table):
  industries = {}
  tickers = []
  company_names = []
  company_industries = []
  #this iteration obtains all the tickers, company_names and industries which have tickers tagged to them
  for row in table.findAll('tr')[1:]:
      ticker = row.findAll('td')[0].text
      company_name = row.findAll('td')[1].text.strip()
      industry = row.findAll('td')[2].text

      if industry not in industries:
        industries[industry] = []
      industries.get(industry).append((ticker, company_name))

  #these 2 lines of code update the industries dictionary with the tickers of the relevant companies of those industries
  for industry in industries.keys():
    curr_industry = [(ticker.replace('\n', ''), company_name) for ticker, company_name in industries[industry]]
    industries[industry] = pd.DataFrame({'Symbol': map(lambda company: company[0], curr_industry), 'Company Name': map(lambda company: company[1], curr_industry)})
  
  return industries
  #print(holdings_df.head())
SP500_scraper(table)


{'Industrials':    Symbol      Company Name
 0     MMM                3M
 1     AOS       A. O. Smith
 2     ADP               ADP
 3     ALK  Alaska Air Group
 4    ALLE          Allegion
 ..    ...               ...
 69   VRSK            Verisk
 70    WAB            Wabtec
 71     WM  Waste Management
 72    GWW    W. W. Grainger
 73    XYL        Xylem Inc.
 
 [74 rows x 2 columns],
 'Health Care':    Symbol                  Company Name
 0     ABT                        Abbott
 1    ABBV                        AbbVie
 2       A          Agilent Technologies
 3    ALGN              Align Technology
 4    AMGN                         Amgen
 ..    ...                           ...
 60   VTRS                       Viatris
 61    WAT            Waters Corporation
 62    WST  West Pharmaceutical Services
 63    ZBH                 Zimmer Biomet
 64    ZTS                        Zoetis
 
 [65 rows x 2 columns],
 'Information Technology':    Symbol        Company Name
 0     ACN           

In [31]:
def data_obtainer(tickers, month_delta, len_moving_window, data_start, data_end):
    data_end += relativedelta(months=-month_delta)
    data_start = data_end + relativedelta(months=-len_moving_window)

    df = yf.download(tickers, start=data_start, end=data_end)
    df = df[['Close']]
    df = df.apply(np.log).diff()

    return df

industries = SP500_scraper(table)
for industry in industries.keys():
    curr_industry = industries[industry]
    tickers = curr_industry['Symbol'].tolist()
    month_delta = 3  # Example number of months to subtract from data_end
    len_moving_window = 6  # Example length of the moving window
    data_start = pd.to_datetime('2022-01-01')  # Example start date
    data_end = pd.to_datetime('2022-12-31')  # Example end date
    industries[industry] = data_obtainer(tickers, month_delta, len_moving_window, data_start, data_end)

for industry in industries.keys():
    print(industries[industry])

[*********************100%%**********************]  74 of 74 completed
[*********************100%%**********************]  65 of 65 completed


1 Failed download:
['GEHC']: Exception("%ticker%: Data doesn't exist for startDate = 1648612800, endDate = 1664510400")



[*********************100%%**********************]  67 of 67 completed
[*********************100%%**********************]  23 of 23 completed
[*********************100%%**********************]  38 of 38 completed


2 Failed downloads:
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2022-03-30 00:00:00 -> 2022-09-30 00:00:00)')
['KVUE']: Exception("%ticker%: Data doesn't exist for startDate = 1648612800, endDate = 1664510400")



[*********************100%%**********************]  30 of 30 completed
[*********************100%%**********************]  72 of 72 completed


1 Failed download:
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')



[*********************100%%**********************]  52 of 52 completed
[*********************100%%**********************]  29 of 29 completed
[*********************100%%**********************]  30 of 30 completed
[*********************100%%**********************]  23 of 23 completed
               Close                                                    \
                 AAL       ADP       ALK      ALLE       AME       AOS   
Date                                                                     
2022-03-30       NaN       NaN       NaN       NaN       NaN       NaN   
2022-03-31  0.011019 -0.000966  0.000000 -0.032002 -0.016753 -0.036273   
2022-04-01 -0.000548  0.025856 -0.001035  0.015726  0.008970  0.030672   
2022-04-04 -0.000548  0.007169 -0.013726 -0.001975 -0.002832  0.017753   
2022-04-05 -0.021625  0.011920 -0.005086 -0.010568 -0.016630 -0.025218   
...              ...       ...       ...       ...       ...       ...   
2022-09-23 -0.040134 -0.000825 -0.026617  0.01543